# O que é Termooo



[Termo](http://term.ooo), uma versão tupi-guarani do [Wordle](https://www.nytimes.com/games/wordle/index.html), é um jogo de advinhação que tem como objetivo encontrar uma "palavra secreta". De início, sabemos apenas que ela faz parte da lingua portuguesa (pt-BR) e que tem cinco letras, o que nos dá uma gama enorme de possibilidades (_em nosso léxico, mais de 3000 palavras_). Temos seis "chutes" para conseguir acertar a palavra e, a cada chute, recebemos pistas: o jogo nos dirá, para cada letra, se acertamos (_a palavra tem aquela letra naquela posição_) se quase acertamos (_a palavra tem aquela letra, mas não naquela posição_), ou se erramos (_a palavra não tem aquela letra_).

Intuitivamente, segue-se um processo de eliminação para deduzir qual é a palavra. As primeiras tentativas servem para ter um número mínimo de pistas (letras) e, a partir daí, fazemos um esforço mental para lembrar de alguma palavra que obedeça as **restrições** impostas pelo jogo.

Para resolver esse jogo de forma algoritimica, vamos emular o que um ser humano faria, mas de forma mais exata e precisa; como não contamos com a intuição, teremos que usar outras capacidades que o computador nos trás.

# Resolvendo Termo de forma programática



Ao longo deste notebook vamos descrever uma abordagem para encontrar a palavra no menor número de chutes possíveis. Idealmente, queremos um algoritmo que, na média, seja mais eficiente que um ser humano. 

Para iniciar, utilizaremos um léxico com as palavras que entendemos serem possíveis. O autor do jogo publicou o léxico utilizado pelo próprio Termo, mas vamos utilizar uma fonte diferente, para provar que o algoritmo é eficiente (usar o léxico do jogo reduz o espaço de busca e torna mais fácil encontrar a palavra).

Daí em diante, vamos utilizar uma abordagem estatística para encontrar a melhor palavra. O que queremos é reduzir o espaço de busca para apenas 1 palavra no menor número de chutes possíveis, então vamos precisar do seguinte:

* Determinar quão bom é um chute.
* Filtrar o léxico (*espaço de busca*) com base nas pistas que já temos.

Não é a proposta aqui de desenvolver um programa que _jogue_ o jogo. Isso poderia ser feito, por exemplo, através de um scriptlet (quem sabe meu próximo projeto?). Por enquanto, só queremos achar uma abordagem que tenha um bom desempenho. Para isso, um notebook Jupyter é ótimo.

# Primeiros passos

Aqui carregamos algumas bibliotecas e configuramos alguns parâmetros genéricos.

In [47]:
from typing import List
from unidecode import unidecode

In [48]:
WORD_LENGTH = 5
"""Tamanho da palavra a ser procurada"""

'Tamanho da palavra a ser procurada'

# Carregar o Léxico



Precisamos de uma lista de palavras válidas para a língua portuguesa - chamamos isso de léxico. Vamos utilizar um produzido por um time de especialistas da USP, no projeto [Unitex](http://www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/dicionarios.html).

Esse léxico possui mais de 75.000 palavras encontradas em textos em português. 

> ⚠ Uma análise do arquivo irá mostrar vários verbetes que não seriam válidos no term.ooo, como palavras em inglês (Windows ou wheel), pois esse léxico foi extraído de textos reais onde verbetes em outras línguas também são encontrados. Futuramente, poderíamos fazer uma redução dessa base para apenas palavras usuais.

Note que carregamos apenas palavras com 5 caracteres, uma vez que esse tamanho é fixo no `term.ooo`.

In [49]:
def load_dictionary(filename: str, word_length: int):
    """
    Loads a lexicon from a file, where the each word is the first token on a line.
    Tokens are separated by commas.
    """
    with open(filename, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.split(',')[0]
            line = line.strip()

            if len(line) == word_length:
                yield line


Removemos diátricos (não são necessários no jogo). Usamos `set` para remover palavras repetidas que surgem após a remoção.

In [50]:
words = map(unidecode, load_dictionary('./DELAS_PB.dic', WORD_LENGTH))

def remove_duplicates(words: List[str]) -> List[str]:
	"""
	Removes duplicates from a sorted list.
	"""
	result = []

	previous = None

	for word in words:
		if word != previous:
			result.append(word)
			previous = word

	return result
		
words = remove_duplicates(words)


Também removemos do léxico palavras que já sabemos que não são aceitas no jogo.

In [51]:
forbidden = list(load_dictionary('forbidden_words.dic', WORD_LENGTH))

words = list(filter(lambda word: word not in forbidden, words))


In [52]:
words

['aarao',
 'aaron',
 'abaca',
 'abaco',
 'abada',
 'abade',
 'abalo',
 'abano',
 'abate',
 'abati',
 'abbud',
 'abdus',
 'abeca',
 'abeta',
 'abete',
 'abeto',
 'abiel',
 'abita',
 'abner',
 'aboar',
 'aboio',
 'abono',
 'abrao',
 'abreu',
 'abril',
 'abrir',
 'abuso',
 'abuta',
 'acaca',
 'acaju',
 'acapu',
 'acara',
 'acari',
 'acaro',
 'acaso',
 'acato',
 'acaua',
 'aceno',
 'aceso',
 'achar',
 'acido',
 'acima',
 'acola',
 'acuar',
 'acude',
 'adaga',
 'adail',
 'adair',
 'adams',
 'adega',
 'adelo',
 'adeus',
 'adiar',
 'adido',
 'admir',
 'adobe',
 'adolf',
 'aduar',
 'adubo',
 'adufa',
 'adufe',
 'advir',
 'aecio',
 'aereo',
 'afago',
 'afear',
 'afeto',
 'afiar',
 'afilo',
 'afixo',
 'aflar',
 'afogo',
 'afono',
 'afora',
 'afoxe',
 'agamo',
 'agape',
 'agata',
 'agave',
 'agnes',
 'agogo',
 'agora',
 'agrar',
 'aguai',
 'aguar',
 'agude',
 'agudo',
 'aguia',
 'aidar',
 'ainda',
 'aipim',
 'airar',
 'aires',
 'ajeru',
 'ajuda',
 'ajuri',
 'alado',
 'alain',
 'alamo',
 'albor',


# Classificando um "chute"


## Introdução

Para jogar o Termo, precisamos de um chute inicial. Essa primeira escolha é muito importante, pois queremos eliminar o maior número possível de palavras. Vamos estudar alguns chutes em um léxico fictício.


> ### Exemplo 1
> 
> Imagine um léxico de 5 palavras de tamanho 3, e um alfabeto de 3 letras: 
> 
> ``` python
> ['CBA', 'BBC', 'CAC', 'BCA', 'CAB']
> ```
> 
> Se a palavra escolhida for 'BBC' e chutássemos 'BCA', iamos ter uma informação:
> 
> 
> | B | C | A |
> |:-:|:-:|:-:|
> |🟢 |🟡| 🔴 |
> 
> Com essas pistas, podemos descartar as palavras do léxico que não começam com B...
> 
> ``` python
> ['BBC', 'BCA']
> ```
> 
> ...e descartar todas que tem A...
> ``` python
> ['BBC']
> ```
> ... o que nos leva a respota correta!

O ideal então é realizar um chute que remova o maior número de palavras. Em nosso exemplo foram eliminadas 4 palavras de 5. Esse seria o ideal, mas veja o seguinte: vamos fazer o mesmo chute, mas considerar que a palavra escolhida foi `BBC`:

> ### Exemplo 2
> 
> Léxico:
> 
> ``` python
> ['CBA', 'BBC', 'CAC', 'BCA', 'CAB']
> ```
>
> Chute:
> 
> | B | C | A |
> |:-:|:-:|:-:|
> |🟡 |🟡| 🔴 |
>
> Eliminamos todas as palavras que não tem B:
>
> ``` python
> ['CBA', 'BBC', 'BCA', 'CAB']
> ```
>
> E agora todas que tem B na primeira posição:
>
> ``` python
> ['CBA', 'CAB']
> ```
> 
> Agora eliminamos as que não tem C:
>
> ``` python
> ['CBA', 'CAB']
> ```
> 
> E, finalmente, as que tem C na segunda posição:
> 
> ``` python
> ['CBA', 'CAB']
> ```
>
> Eliminamos apenas 3 palavras!
>

A eficácia do "chute" varia não só com o léxico, mas também com base na palavra que foi escolhida pelo jogo. Então, na realidade, existe uma **probabilidade** de quantas palavras são eliminadas por um chute, que varia conforme a palavra secreta.

Se assumirmos que a probablidade o jogo escolher uma palavra de nosso léxico é [uniforme](https://pt.wikipedia.org/wiki/Distribui%C3%A7%C3%A3o_uniforme), i.e, P(W=w) = 1/|W|, para todo w ∈ W, poderíamos calcular a _qualidade_ de usar uma palavra w¹ como chute contabilizando, para cada possibilidade de palavra secreta (todo w ∈ W), quantas palavras são eliminadas. Multiplicando esse valor pela probabilidade do segredo ser aquela palavra (1/|W|), teremos um número que representa a capacidade de eliminação de palavras daquele chute.

> ⚠ Note que isso significa que para cada chute, temos que testar todas as palavras para ver se serão eliminadas, um algoritmo O(n²), que terá baixa performance em léxicos grandes. Mais tarde, iremos estudar como otimizar esse algoritmo.

# Coding up
Um chute (`guess`) deve gerar pistas (`clue`s) com base em um segredo (`secret`).

In [53]:
# Modelamos um resultado de um chute:
from enum import Enum

class Clue(Enum):
	"""The word does not contains the letter."""
	NOT_CONTAINS = 0
	"""The word contains the letter but no in the informed position."""
	CONTAINS = 1
	"""The word contains the letter and in the informed position."""
	CORRECT = 2

Clues = List[Clue]

In [54]:
def check_guess(guess: str, word: str) -> Clues:
	"""
	Match two words and return the clues.
	"""
	result = []
	
	if len(guess) != len(word):
		raise ValueError('The guess and the word must have the same length.')
		
	for i, letter in enumerate(guess):
		if letter == word[i]:
			result.append(Clue.CORRECT)
		elif letter in word:
			result.append(Clue.CONTAINS)
		else:
			result.append(Clue.NOT_CONTAINS)
	return result

In [55]:
assert check_guess("abcd", "abcd") == [Clue.CORRECT] * 4
assert check_guess("abcd", "abce") == [Clue.CORRECT] * 3 + [Clue.NOT_CONTAINS]
assert check_guess("abcd", "dcba") == [Clue.CONTAINS] * 4

In [56]:
def is_match(word: str, guess: str, guess_result: Clues) -> bool:
	"""
	Checks if the word matches the guess.
	"""
	for i, guess_letter in enumerate(guess):
		if guess_result[i] == Clue.NOT_CONTAINS:
			if guess_letter in word: # suboptimal - this is scanning the whole word again
				return False
		elif guess_result[i] == Clue.CONTAINS:
			if guess[i] == guess_letter or guess_letter not in word:
				return False
		elif guess_result[i] == Clue.CORRECT:
			if guess_letter != word[i]:
				return False
		else:
			raise ValueError(f'Invalid guess result {guess_result[i]}')

	return True

In [57]:
def guess_quality(lexicon: List[str], guess: str) -> float:
    """
    Returns the quality of a guess based on the lexicon.
    """
    score = 0  # number of letters removed

    for secret in lexicon:
        clues = check_guess(guess, secret)
        print(f"Clues for {guess} against {secret}: {clues}")
        for word in lexicon:
            if not is_match(word, guess, clues):
                score += 1

    return score / len(lexicon)


In [58]:
def best_guess(lexicon: List[str]) -> str:
    """
    Returns the best guess based on the lexicon.
    """
    best_guess = None
    best_quality = float('inf')
    
    for guess in lexicon:
        quality = guess_quality(lexicon, guess)
        
        print(f"Quality of '{guess}' is {quality}.")

        if quality > best_quality:
            best_guess = guess
            best_quality = quality

    return best_guess

In [59]:
# Too slow
# guess = best_guess(words)

## Otimizando

Se você tentou executar o código acima, está fácil de perceber que precisamos otimizá-lo. O(N³) é impossível de processar mesmo para N ~ 3000. 

Temos alguma opções. Um jeito força bruta seria utilizar várias threads - o código acima é facilmente paralelizável, afinal, ele não altera dados em memória e cada resultado é independende dos demais. Mas isso não estaria baixando a complexidade do algorítimo, apenas diminuindo o tempo de execução. É mais interessante, primeiro, explorarmos outras opções.

### Diminuindo as combinações

A maior complexidade vem de percorrermos o léxico 3 vezes, aninhadamente: uma para determinar o chute para o qual queremos calcular a qualidade, um para determinar o segredo, e um que escolhe a palavra alvo.

``` python
    for guess in lexicon:
        for secret in lexicon:
            clues = check_guess(guess, secret)
            for word in lexicon:
                if not is_match(word, guess, clues):
                    score += 1
```

Incrementamos o `score` quando `is_match(word, guess, clues)`, e é nele que estamos interessados. Porém, essa chamada não utiliza o `secret` diretamente - ele é utilizado de forma indireta, para a geração das clues. Será que conseguimos as clues de outra forma? Isso evitaria percorrer o léxico mais uma vez e efetivamente traria nosso algoritmo para O(N²)!

Poderíamos, por exemplo, ter a lista completa de "clues" possíveis... por exemplo, temos o "acerto", que é uma sequência de 5 `CORRECT`s, temos um quase certo com 4 `CORRECT`s e 1 `NOT_CONTAINS`, ou então 3 `CORRECT` e 1 `CONTAINS` e 1 `NOT_CONTAINS`... etc. Se quisessemos enumerar todas essas, precisarímos de menos de 3⁵ posições. Digo menos, pois algumas combinações não seriam possíveis, por exemplo, 4 `CORRECT`s e 1 `CONTAIN`s. Mas se ignoramos isso, podemos simplesmente gerar todas as combinações de clues possíveis e substituir o tratamento do secret.   


In [60]:
def all_clues(length: int) -> List[Clues]:
	"""
	Returns all possible clues.
	"""
	if length == 0:
		return []

	if length == 1:
		return [[Clue.CONTAINS], [Clue.NOT_CONTAINS], [Clue.CORRECT]]
	
	clues = []
	for clue in all_clues(length - 1):
		clues.append(clue + [Clue.CONTAINS])
		clues.append(clue + [Clue.NOT_CONTAINS])
		clues.append(clue + [Clue.CORRECT])

	return clues

possible_clues = all_clues(5)


Tendo esta lista completa de clues, podemos então melhorar `guess_quality`:

In [61]:
def guess_quality(lexicon: List[str], guess: str) -> float:
    """
    Returns the quality of a guess based on the lexicon.
    """
    score = 0  # number of letters removed

    for clues in possible_clues:
        for word in lexicon:
            if not is_match(word, guess, clues):
                score += 1

    return score / len(lexicon)


In [62]:
# still too slow
#guess = best_guess(words)
#guess

### Agrupando as remoções

As melhorias do passo anterior diminuem bastante o tempo de processamento, mas ainda assim, para uma quantidade elevada de palavras, o tempo de execução em uma única thread é longo. 

Uma maneira de diminuir o tempo de execução é agrupar as palavras no léxico de maneira que seja mais fácil de filtrar pelo clue. Vamos utilizar um atributo formado que indica quais letras estão contidas na palavra, que é fácil de armazenar em uma variável de 32 bits, afinal, só temos 26 letras no alfabeto. 

> ! TODO: Reescrever explicação



In [63]:
from typing import Dict, Iterable


class IndexedLexicon:
	"""A lexicon of a language."""
	def __init__(self, words: Iterable[str]) -> None:
		self._buckets: Dict[int, List[str]] = {}

		for word in words:
			idx = self._to_bitmask(word)
			
			if idx not in self._buckets:
				self._buckets[idx] = []

			self._buckets[idx].append(word)

	def _to_bitmask(self, word: str) -> int:
		buffer = ['0'] * 26
		
		for letter in word.lower():
			buffer[ord(letter) - ord('a')] = '1'

		return int(''.join(buffer), 2)

	def filter(self, clues: Clues, guess: str) -> Iterable[str]:
		for word in self._enumerate_filtered(clues, guess):
			if self._is_match_position(word, guess, clues):
				yield word

	def _enumerate_filtered(self, clues: Clues, guess: str) -> Iterable[str]:
		contains = ""
		
		for letter, result in zip(guess, clues):
			if result == Clue.CONTAINS or result == Clue.CORRECT:
				contains += letter

		bitmask_contains = self._to_bitmask(contains)

		not_contains = ""

		for letter, result in zip(guess, clues):
			if result == Clue.NOT_CONTAINS:
				not_contains += letter

		bitmask_not_contains = self._to_bitmask(not_contains)

		for idx in self._buckets:
			if (idx & bitmask_contains) == bitmask_contains and (idx & bitmask_not_contains) == 0:
				for word in self._buckets[idx]:
					yield word		

	def _is_match_position(self, word: str, guess: str, guess_result: Clues) -> bool:
		"""
		Checks if the word matches the guess.
		"""
		
		for letter, guess_letter, result in zip(word, guess, guess_result):
			if result == Clue.NOT_CONTAINS:
				continue 
			elif result == Clue.CONTAINS:
				if letter == guess_letter:
					return False
			elif result == Clue.CORRECT:
				if guess_letter != letter:
					return False
			else:
				raise ValueError(f'Invalid guess result {result}')

		return True
	
	def __len__(self):
		l = 0

		for words in self._buckets.values():
			l += len(words)

		return l

	def __iter__(self):
		for words in self._buckets.values():
			for word in words:
				yield word

# Tests
lexicon = IndexedLexicon(["AA", "BB", "CC", "AC", "BA"])

print(len(lexicon) )
assert list(lexicon) == ["AA", "BB", "CC", "AC", "BA"]
assert len(lexicon) == 5

assert lexicon._to_bitmask("z") == 1
assert len(format(lexicon._to_bitmask("a") , "b")) == 26

assert list(lexicon._enumerate_filtered([Clue.CONTAINS, Clue.NOT_CONTAINS], "AC")) == ["AA", "BA"]
assert list(lexicon.filter([Clue.CONTAINS, Clue.CONTAINS], "AB")) == ["BA"]

5


In [64]:
words = IndexedLexicon(words)

In [70]:
number_of_words = len(words)

best_score = 0
best_guess = None

for guess in words:
	score = 0
	
	for clues in possible_clues:
		filtered = words.filter(clues, guess)
		score += number_of_words - len(list(filtered))

	print(f"Score for {guess} is {score}")
	
	if score > best_score:
		best_score = score
		best_guess = guess

best_guess

Score for aarao is 861278
Score for rorar is 861278
Score for aaron is 861278
Score for arnon is 861278
Score for ornar is 861278
Score for abaca is 861278
Score for abaco is 861278
Score for abada is 861278
Score for abade is 861278
Score for abalo is 861278
Score for balao is 861278
Score for lobao is 861278
Score for abano is 861278
Score for abono is 861278
Score for abate is 861278
Score for abeta is 861278
Score for abete is 861278
Score for baeta is 861278
Score for beata is 861278
Score for abati is 861278
Score for abita is 861278
Score for baita is 861278
Score for abbud is 861278
Score for abdus is 861278
Score for abeca is 861278
Score for abeto is 861278
Score for beato is 861278
Score for betao is 861278
Score for boate is 861278
Score for abiel is 861278
Score for baile is 861278
Score for biela is 861278
Score for abner is 861278
Score for berna is 861278
Score for aboar is 861278
Score for abrao is 861278
Score for barao is 861278
Score for barro is 861278
Score for bo

'aarao'

In [ ]:
def filter_by_guess(lexicon: List[str], guess: str, guess_result: List[Clue]) -> List[str]:
    """
    Filter the lexicon by the guess result.
    """
    
    
    """
    Returns the lexicon filtered by the guess based on the result.
    """
    for word in lexicon:
        if is_match(guess, guess_result, word):
            yield word


assert list(filter_by_guess(['AB', 'AC', 'BA', 'BC', 'CA', 'CB'], 'BA', [Clue.CORRECT, Clue.NOT_CONTAINS])) == ['BC']
assert list(filter_by_guess(['AB', 'AC', 'BA', 'BC', 'CA', 'CB'], 'BA', [Clue.NOT_CONTAINS, Clue.CORRECT])) == ['CA']
assert list(filter_by_guess(['AB', 'AC', 'BA', 'BC', 'CA', 'CB'], 'BA', [Clue.CORRECT, Clue.CORRECT])) == ['BA']


# Primeiro chute

Agora, vamos encontrar a palavra para ser nosso melhor primeiro "chute". Considerando uma distribuição de palavras que sigam o léxico (algo que não é, necessariamente, verdade ☹), vamos verificar quais são as letras mais frequentes. Dessa forma, vamos fazer o primeiro chute com uma palavra que contenha as letras mais frequentes na língua.

Note que as cinco letras mais frequentes podem não formar uma palavra válida. Por isto, vamos pegar a palavra do léxico que possua o maior número de letras frequentes.

Primeiro, nós contabilizamos as letras no léxico.

In [ ]:
letters = {}

for word in words:
    for letter in word:
        if letter in letters:
            letters[letter] += 1
        else:
            letters[letter] = 1

# Sort by most common letter
sorted_letters = sorted(letters.items(), key=lambda x: x[1], reverse=True)
sorted_letters = list(map(lambda x: x[0], sorted_letters))
'Ranked letters: ' + ', '.join(sorted_letters)

Agora, procuramos uma palavra no léxico que maximize as letras frequentes. Um ponto importante aqui é que não queremos repetições - i.e., não queremos uma palavra que tenha letras repetidas.

In [ ]:
def find_first_word_with_letters(words: List[str], letters: List[str]) -> str:
    """
    Finds the *first* word in the list that is made only of letters in the list.
    """
    for word in words:
        target = letters.copy()  # what letters are left to find

        for letter in word:
            if letter in target:
                target.remove(letter)

        if len(target) == len(letters) - len(word):
            return word

    return None


def find_next_best_guess(words, ranked_letters, guesses=[]):
    """
    Finds the best next guess based on the ranked letters.

    words - list of all possible words in the game
    ranked_letters - list of letters sorted by most common
    guesses - list of guesses made so far
    """

    # Build a list of all letters that already have been guessed
    guessed_letters = set([list(guess) for guess in guesses])               # list(str) split str by chars...

    # Filter the ranked letters to only include non-guessed letters
    if len(guessed_letters) != 0:
        print(f"Skipping letters: {guessed_letters}")        
        ranked_letters = filter(
            lambda letter: letter not in guessed_letters, ranked_letters)
        ranked_letters = list(ranked_letters)

    for size in range(WORD_LENGTH, len(ranked_letters)):
        letters = ranked_letters[:size]
        guess = find_first_word_with_letters(words, letters)

        if guess != None:
            return guess
        else:
            joined_letters = ''.join(letters)

            msg = f"Could not find a word using the top letters '{joined_letters}'. " #+ \
                #f"Only {len(ranked_letters)} letters are available."

            print(msg)


best_guess = find_next_best_guess(words, sorted_letters)
best_guess


# Jogando o jogo

Com nosso primeiro chute em mãos, iniciamos o jogo. Precisamos de uma maneira para codificar a resposta que o jogo nos da para cada chute; utilizaremos uma string para isso, onde cada posição identifica o retorno do jogo, da seguinte maneira:

- Um espaço denota uma falha.
- Um caractere `i` indica que a letra está inclusa na palavra, mas não na posição correta.
- Um caracter `c` indica que a letra está inclusa na palavra e na posição correta.

Para jogar, informamos ao usuário qual deve ser o chute, e aguardamos que ele insira a resposta recebida. Aí computamos o melhor chute considerando o que já sabemos.

O melhor chute é aquele que obedece aquilo que já sabemos das palavras (por exemplo, quais letras devem estar presentes) e que prefere utilizar letras mais frequentes.

> Seguimos essa heurística por que não temos como deduzir a probabilidade de uma palavra aparecer no jogo, mas temos uma regra para determinar a probabilidade de uma palavra incluir uma letra. 

In [ ]:
def validate_result(result: str) -> bool:
	if len(result) != 5:
		print('Result must be 5 letters long')
		return False
	
	if any(c not in ' ci' for c in result):
		print('Result must be composed of spaces, "i" or "c"')	
		return False

	return True

In [ ]:
def describe_next_guess(included: List[str], not_included: List[str], known: List[str]) -> None:
    print('Guess must include {}'.format(', '.join(included)))
    print('Guess must not include {}'.format(', '.join(not_included)))
    print('Guess must be like {}'.format(', '.join(known)))


Para criar um novo chute, primeiro recuperamos a lista de palavras e a filtramos pelas regras.


In [ ]:
def match(word: str, like: List[str], unlike: List[List[set]]) -> bool:
	for i in range(len(word)):
		if like[i] != ' ' and word[i] != like[i]:
			return False
		
		if word[i] in unlike[i]:
			return False

	return True


def filter_by(words: List[str], included_letters: List[str], not_included_letters: List[str], like: List[str], unlike: List[List[set]]):
	for word in words:
		# Check if any letter of the word is in the forbidden list
		if any(letter in not_included_letters for letter in word):
			continue
		# Check if any letter from the must-have list is missing
		if any(letter not in word for letter in included_letters):
			continue
		
		if not match(word, like, unlike):
			continue

		yield word		
	

In [ ]:
def play_game(guesses: List[str], words: List[str], ranked_letters: List[str]):
    all_words = words.copy()

    # what letters should be in the word
    included_letters = set()

    # what letters should not be in the word
    not_included_letters = set()

    # the word that is being guessed, every letter is blank until it is known
    final_word = [letter for letter in '     ']

    # what letters must not be in each position in the word
    unlike = [set() for _ in range(5)]

    while True:
        print(f"Currently there are {len(words)} possible words.")

        result = input(
            f'Try the guess "{guesses[-1]}". What was the result? ').lower()

        if len(result) == 0:
            print("Empty result. Stopping game.")
            return

        if not validate_result(result):
            continue

        for i, r in enumerate(result):
            letter = guesses[-1][i]

            if r == ' ':
                not_included_letters.add(letter)
            elif r == 'c':
                final_word[i] = letter
            elif r == 'i':
                included_letters.add(letter)
                unlike[i].add(letter)

        #describe_next_guess(included_letters, not_included_letters, final_word)

        # update list of possible words based on the current state
        words = list(filter_by(words, included_letters,
                     not_included_letters, final_word, unlike))

        if len(words) == 0:
            print("Could not find any words that satisfy parameters. Stopping game.")
            break

        elif len(words) <= 10:
            print("Possible words: " + ', '.join(words))
            break

        else:
            # find the best guess
            guesses.append(find_best_guess(all_words, ranked_letters, guesses))


play_game([best_guess], words, sorted_letters)
